# 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻语料，该语料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

## Step1:数据分析
请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [19]:
fname = 'D:/code_project/datasource/export_sql_1558435/sqlResult_1558435.csv'
import pandas as pd
contents = pd.read_csv(fname, encoding='gb18030')
contents.head(10)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
6,89611,NaN,威锋网@http://www.feng.com/,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""21""...",配骁龙660 全面屏鼻祖夏普新机酝酿中,http://www.cnbeta.com/articles/tech/623603.htm
7,89610,申玉彬 整理,中国证券报?中证网,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
8,89609,李杭_BJS4645,荆楚网-楚天都市报,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1600""...",武汉千余警察出动 抓获808名俊男靓女全是诈骗犯,http://news.163.com/17/0614/14/CMT9N8G80001899...
9,89608,吴瞬,中国证券报?中证网,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...


In [8]:
xinhua_news = contents[contents['source']==u'新华社']
len(xinhua_news)/len(content)

0.8778051801676133

## Step2: 数据预处理
将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [33]:
source = [1  if contents['source'][i]==u'新华社' else 0 for i in range(len(contents['source']))]
content = contents['content']

In [34]:
data = {'content':content,'source':source}

In [35]:
df = pd.DataFrame(data,columns=['content','source'])
df.fillna('') # fillna的作用是填充缺失值
df.head(10)

,content,source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0
5,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,0
6,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,0
7,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...,0
8,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,0
9,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,0


## Step3: 使用tfidf进行文本向量化
参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 
对文本进行向量化

数据清洗

In [70]:
import jieba
import re

def cut(string):
    words = jieba.cut(string)
    return ' '.join(words)

def token(string): #替换非字符的内容
    return re.compile(r'[^\d\w\.\%]+').sub(' ',string)

In [71]:
token('this is 一个测试///')

'this is 一个测试 '

In [72]:
content_str = df['content'].tolist()

In [73]:
print(content_str[5])
token(content_str[5])

　　受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金融股和白马股表现喜人，但是尾盘跳水之后，仅金融板块仍维系红盘状态。分析人士认为，金融股受益于MSCI纳入A股和低估值而重获资金青睐，但是存量资金博弈格局下，风格交替的震荡格局料延续。流动性改善、经济悲观预期修正等有助于支撑板块继而大盘指数逐步向好。
　　“一九”再现
　　周四，A股市场未能延续周三的上行态势，两市成交小幅放量。29个中信一级行业中，收盘仅银行和非银行金融两个行业指数收红，分别上涨1.80%和0.20%。
　　从二级行业来看，股份制与城商行的涨幅最高，达到2.22%，国有银行上涨0.82%，信托及其他上涨0.64%，保险板块上涨0.34%，证券板块上涨0.06%。
　　银行板块25只成分股中，共有21只收红。其中，招商银行涨幅最大，上涨6.66%，贵阳银行上涨3.65%，上海银行、华夏银行、浦发银行和兴业银行的涨幅均超过1.50%。非银行金融板块44只成分股中，共17只个股上涨。其中，安信信托、中国太保涨幅居前两名，分别上涨4.57%和3.04%，西水股份、华安证券、中国人寿和新华保险的涨幅也均超过2%。相对而言，券商股多小幅下跌。
　　近期，对A股市场消息面影响最大的就是MSCI宣布从2018年6月开始将A股纳入MSCI新兴市场指数。而其中，金融股是占比最大的一个群体。国金证券李立峰团队指出，最新方案中包含的222只成分股中，剔除了中等市值、非互联互通可交易的股票以及有停牌限制的标的，由于纳入了很多大市值AH股，A股在MSCI EM中的权重由0.5%上升到了0.73%。其中，金融板块占比最高，达到40.11%，泛消费次之，占比为24.26%，两个板块涵盖了大部分权重股。动态来看，由于加入了很多是指占比高的金融公司，金融板块的权重增加了近一半，其他大部分行业权重都受到了稀释。
　　尽管A股被纳入MSCI这一利好事件对短期市场情绪有所提振、对中长期海外增量资金预期升温，但短期内，市场量能尚不能有效放大，金融股独乐乐情景也就难以持续。存量博弈格局下，风格交替、指数震荡格局难改变。
　　光大证券指出，利好并未引起市场太大的热情，两市指数和成交量均较为平淡，但市场风格出现了较大变化，白马股金融股上涨的同时，成长股、题材股全天低迷。这表明市场增量资金依然很少，存量资金在不

' 受到A股被纳入MSCI指数的利好消息刺激 A股市场从周三开始再度上演龙马行情 周四上午金融股和白马股表现喜人 但是尾盘跳水之后 仅金融板块仍维系红盘状态 分析人士认为 金融股受益于MSCI纳入A股和低估值而重获资金青睐 但是存量资金博弈格局下 风格交替的震荡格局料延续 流动性改善 经济悲观预期修正等有助于支撑板块继而大盘指数逐步向好 一九 再现 周四 A股市场未能延续周三的上行态势 两市成交小幅放量 29个中信一级行业中 收盘仅银行和非银行金融两个行业指数收红 分别上涨1.80%和0.20% 从二级行业来看 股份制与城商行的涨幅最高 达到2.22% 国有银行上涨0.82% 信托及其他上涨0.64% 保险板块上涨0.34% 证券板块上涨0.06% 银行板块25只成分股中 共有21只收红 其中 招商银行涨幅最大 上涨6.66% 贵阳银行上涨3.65% 上海银行 华夏银行 浦发银行和兴业银行的涨幅均超过1.50% 非银行金融板块44只成分股中 共17只个股上涨 其中 安信信托 中国太保涨幅居前两名 分别上涨4.57%和3.04% 西水股份 华安证券 中国人寿和新华保险的涨幅也均超过2% 相对而言 券商股多小幅下跌 近期 对A股市场消息面影响最大的就是MSCI宣布从2018年6月开始将A股纳入MSCI新兴市场指数 而其中 金融股是占比最大的一个群体 国金证券李立峰团队指出 最新方案中包含的222只成分股中 剔除了中等市值 非互联互通可交易的股票以及有停牌限制的标的 由于纳入了很多大市值AH股 A股在MSCI EM中的权重由0.5%上升到了0.73% 其中 金融板块占比最高 达到40.11% 泛消费次之 占比为24.26% 两个板块涵盖了大部分权重股 动态来看 由于加入了很多是指占比高的金融公司 金融板块的权重增加了近一半 其他大部分行业权重都受到了稀释 尽管A股被纳入MSCI这一利好事件对短期市场情绪有所提振 对中长期海外增量资金预期升温 但短期内 市场量能尚不能有效放大 金融股独乐乐情景也就难以持续 存量博弈格局下 风格交替 指数震荡格局难改变 光大证券指出 利好并未引起市场太大的热情 两市指数和成交量均较为平淡 但市场风格出现了较大变化 白马股金融股上涨的同时 成长股 题材股全天低迷 这表明市场增量资金依然很少 存量资金在不同板块之间腾挪 这样的跷跷板格局使得指数难

In [74]:
content_str = [token(str(n)) for n in content_str] #删除非文本和数字的字符

In [75]:
content_str = [cut(n) for n in content_str] #分词

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HSUHUI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.914 seconds.
Prefix dict has been built succesfully.


In [76]:
content_str[5]

'  受到 A股 被 纳入 MSCI 指数 的 利好 消息 刺激   A股 市场 从 周三 开始 再度 上演 龙马 行情   周四 上午 金融股 和 白马股 表现 喜人   但是 尾盘 跳水 之后   仅 金融 板块 仍 维系 红盘 状态   分析 人士 认为   金融股 受益 于 MSCI 纳入 A股 和 低 估值 而 重获 资金 青睐   但是 存量 资金 博弈 格局 下   风格 交替 的 震荡 格局 料 延续   流动性 改善   经济 悲观 预期 修正 等 有助于 支撑 板块 继而 大盘 指数 逐步 向 好   一 九   再现   周四   A股 市场 未能 延续 周三 的 上行 态势   两市 成交 小幅 放量   29 个 中信 一级 行业 中   收盘 仅 银行 和 非银行 金融 两个 行业 指数 收红   分别 上涨 1.80% 和 0.20%   从 二级 行业 来看   股份制 与 城商行 的 涨幅 最高   达到 2.22%   国有银行 上涨 0.82%   信托 及其 他 上涨 0.64%   保险 板块 上涨 0.34%   证券 板块 上涨 0.06%   银行 板块 25 只 成分股 中   共有 21 只 收红   其中   招商银行 涨幅 最大   上涨 6.66%   贵阳 银行 上涨 3.65%   上海银行   华夏银行   浦发银行 和 兴业银行 的 涨幅 均 超过 1.50%   非银行 金融 板块 44 只 成分股 中   共 17 只 个股 上涨   其中   安信 信托   中国 太保 涨幅 居前 两名   分别 上涨 4.57% 和 3.04%   西水股份   华安 证券   中国 人寿 和 新华 保险 的 涨幅 也 均 超过 2%   相对而言   券商 股多 小幅 下跌   近期   对 A股 市场 消息面 影响 最大 的 就是 MSCI 宣布 从 2018 年 6 月 开始 将 A股 纳入 MSCI 新兴 市场 指数   而 其中   金融股 是 占 比 最大 的 一个 群体   国金 证券 李立峰 团队 指出   最新 方案 中 包含 的 222 只 成分股 中   剔除 了 中等 市值   非 互联互通 可 交易 的 股票 以及 有 停牌 限制 的 标的   由于 纳入 了 很多 大 市值 A H

In [79]:
import codecs
fname2 = 'D:/code_project/datasource/export_sql_1558435/content_cut.txt'
f2 = codecs.open(fname2, 'w', encoding='utf-8')
for n in content_str:
    f2.write(n)
    f2.write(u'\r\n')
f2.close()

文本向量化

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorized = TfidfVectorizer(max_features=1000)
model_tfidf = vectorized.fit(content_str)
tfidf_of_document = model_tfidf.transform(content_str)

In [83]:
tfidf_array = tfidf_of_document.toarray()

In [84]:
print(tfidf_of_document.shape)

(89611, 1000)


## Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型
按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

In [88]:
import numpy as np
random_number = np.random.random(len(tfidf_array[0])) #生成随机一维数组
index = sorted(range(len(tfidf_array[0])), key=lambda x:random_number[x]) #返回排序列表的索引值

数据集划分方法1

In [89]:
data_training = tfidf_array[index[:70000]]
data_validation = tfidf_array[index[70000:80000]]
data_test = tfidf_array[index[80000:]]

数据集划分方法2

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_array, source, test_size=0.3, random_state=42)

构建机器学习模型

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

## Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义.

## Step6: 使用 test_data 对其进行新数据的判断

In [ ]:
y_out = clf.predict(x_test)
clf.score(x_test,y_test)

## Step7: 调整不同的参数，观察变化

## Step8: 不断改变参数，直到性能达到“某个”点。问：“某个点”怎么定义？

初答：当这个模型的acc 大于 0.8778， recall， precision，f1等值都较高的时候。具体查资料

## Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

## Step10： 总结该过程，什么是数据思维？什么是机器学习思维？
(Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。¶

## Step11: 利用lesson课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。¶